## Credit Suite IFRS9 Stress Test Example

The following is an example of how to use the Credit Suite package available on the AWS marketplace. 

First we create some utility functions:

In [47]:
from time import gmtime, strftime
from sagemaker import get_execution_role
from sagemaker.algorithm import AlgorithmEstimator
import sagemaker as sage
from sagemaker.inputs import TrainingInput

def get_sagemaker():
    return sage.Session()

def create_ifrs9_job(sagemaker, 
                        algorithm_arn, 
                        s3_input, 
                        s3_output,
                        reporting_date,
                        loan_tape_path,
                        new_production_path,
                        matrix_path,
                        rating_grades_path,
                        macro_series,
                        macro_params,
                        segment_methodology_path,
                        ecl_max_number_of_periods = 5,
                        number_of_projection_periods = 3
                       ):
    role = get_execution_role()
    training_job_name = "csow-1-" + strftime("%Y-%m-%d-%H-%M-%S", gmtime())
    hyperopts = {
            "reporting_date": reporting_date, # TODO: validate format
            "loan_tape_path": loan_tape_path,
            "new_production_path": new_production_path,
            "matrix_path": matrix_path,
            "rating_grades_path": rating_grades_path,
            "macro_series": macro_series,
            "macro_params": macro_params,
            "segment_methodology_path": segment_methodology_path,
            "ecl_max_number_of_periods": f"{ecl_max_number_of_periods}",
            "number_of_projection_periods": f"{number_of_projection_periods}",
        }
    algo = AlgorithmEstimator(
            sagemaker_session=sagemaker,
            algorithm_arn=algorithm_arn,
            role=role,
            train_instance_count=1,
            train_instance_type='ml.m4.xlarge',
            instance_count=1,
            instance_type="ml.m4.xlarge",
            input_mode='File',
            output_path=s3_output,
            hyperparameters=hyperopts
    )
    algo.fit({'source': s3_input}, job_name=training_job_name)

    print(f"Created ifrs9 job: {training_job_name}")
    return training_job_name

def get_job_description(sagemaker, job_name):
    return sagemaker.describe_training_job(job_name)

def print_job_status(sagemaker, job_name):
    print(f'{job_name}: {get_job_description(sagemaker, job_name)["TrainingJobStatus"]}')

def wait_for_job_completion(sagemaker, job_name):
    print(f"Waiting for completion of {job_name}.")
    r = sagemaker.wait_for_job(job_name)
    description = get_job_description(sagemaker, job_name)
    if description["TrainingJobStatus"] == "Failed":
        message = description["FailureReason"]
        print(f"Ifrs9 job {job_name} failed with the following error: {message}")
    else:
        print(f"Ifrs9 job {job_name} completed.")

### Data definition and execution

SageMaker is framed in terms of supervised machine learning. We have to abuse this terminology to an extent. All of the IFRS9 calculations take place in what SageMaker calls an 'algorithm training job'. Training jobs take input data and produce a 'model'. In our case the contents of the 'model' are the output files from the IFRS9 calculation. SageMaker creates a `.tar.gz` archive of the IFRS9
output files and transfers the archive to a customer-specified S3 location. The archive is always called `model.tar.gz`.

We define s3 locations for data input and output. We also give the correct ARN for credit suite.

We can also define filenames for the particular inputs, adjust the reporting date, and so on.

In [48]:
s3_input = "s3://hpi-hpi30501-sagemaker/dummy_data/Dart_x_v11/"
s3_output = "s3://hpi-hpi30501-sagemaker/sage_out/"
credit_suite_arn = "arn:aws:sagemaker:eu-west-1:985815980388:algorithm/ifrs9-d7e2af4c3c2d4303eb43d3897ac79fd4"

sagemaker = get_sagemaker()

job_name = create_ifrs9_job(
    sagemaker, 
    credit_suite_arn, 
    s3_input, 
    s3_output,
    "2015-12-31",
    "DART_loan_tape.csv",
    "DART_new_production.csv",
    "DART_MM.csv",
    "DART_risk_buckets.csv",
    "DART_macro_data.csv",
    "DART_macro_parameters.csv",
    "DART_segment_methodology.csv",
    5,
    3)
    
print_job_status(sagemaker, job_name)

wait_for_job_completion(sagemaker, job_name)

2021-02-05 13:45:34 Starting - Starting the training job...
2021-02-05 13:45:57 Starting - Launching requested ML instancesProfilerReport-1612532733: InProgress
......
2021-02-05 13:47:00 Starting - Preparing the instances for training......
2021-02-05 13:48:07 Downloading - Downloading input data
2021-02-05 13:48:07 Training - Downloading the training image......
2021-02-05 13:49:02 Uploading - Uploading generated training model
2021-02-05 13:49:02 Completed - Training job completed
INFO:root:Output directory: /opt/ml/model/res-2021-02-05_134850
INFO:csc.data:Reading migration matrix file DART_MM.csv
INFO:csc.data:Reading migration matrix file DART_MM.csv
INFO:csc.data:Reading migration matrix file DART_MM.csv
INFO:csc.data:Reading migration matrix file DART_MM.csv
INFO:root:Processor 3/4 for 8 facilities has started.
INFO:root:Processor 2/4 for 9 facilities has started.
INFO:root:Processor 1/4 for 9 facilities has started.
INFO:root:Processor 4/4 for 8 facilities has started.
INFO:cr